<a href="https://colab.research.google.com/github/henrykohl/Machine-Learning-demo-repo/blob/master/NaturalLanguage/fcc-intro-to-llms/bigram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cpu


In [2]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)


['\n', ' ', '!', '"', '#', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]           ## s 是 字串, encode 是 list
decode = lambda l: ''.join([int_to_string[i] for i in l])      ## l 是 list, decode 是 字串

data = torch.tensor(encode(text), dtype=torch.long)           ## a list of values, value 大小 介於0~82
# print(data[:100])


In [35]:
encoded_hello = torch.tensor(encode('hello'))
decoded_hello = decode(encoded_hello.tolist())
encoded_hello.tolist(),decoded_hello

([63, 60, 67, 67, 70], 'hello')

In [ ]:
n = int(0.8*len(data)) ## n = 186742, 233428
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) ## 詳見下一個cell中的說明
    """ix 的樣態 torch.tensor([i_1, i_2, i_3, i_4])"""
    x = torch.stack([data[i:i+block_size] for i in ix])
    """x 的 shape 是 [4,8]"""
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    """y 的 shape 是 [4,8]"""
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)



inputs:
tensor([[73,  1, 54, 72,  1, 58, 75, 58],
        [58, 69, 65, 62, 58, 57,  1, 28],
        [56, 58, 72, 72, 11,  0,  0,  3],
        [26, 74, 73,  1, 33,  1, 54, 66]], device='cuda:0')
targets:
tensor([[ 1, 54, 72,  1, 58, 75, 58, 71],
        [69, 65, 62, 58, 57,  1, 28, 68],
        [58, 72, 72, 11,  0,  0,  3, 40],
        [74, 73,  1, 33,  1, 54, 66,  1]], device='cuda:0')


字元位置從 0 ~ len(data)-1,所以<font color="red">最後**7個字元**不能選</font>

|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|...|-|-<font color="red">|-|-|-|-|-|-|-|</font>

一個block有八個字元(連續)，所以能取出block的第一個字元位置從 0 ~ len(data)-8

In [36]:
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
  context = x[:t+1]
  target  = y[t]
  print("When input is", context, 'target is', target)

When input is tensor([82]) target is tensor(46)
When input is tensor([82, 46]) target is tensor(63)
When input is tensor([82, 46, 63]) target is tensor(60)
When input is tensor([82, 46, 63, 60]) target is tensor(1)
When input is tensor([82, 46, 63, 60,  1]) target is tensor(42)
When input is tensor([82, 46, 63, 60,  1, 42]) target is tensor(73)
When input is tensor([82, 46, 63, 60,  1, 42, 73]) target is tensor(70)
When input is tensor([82, 46, 63, 60,  1, 42, 73, 70]) target is tensor(65)


In [6]:
"""後面會建立 model = BigramLanguageModel(vocab_size),要在model建立後 estimate_loss()才能使用"""
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters) ## losses 的 shape 是 [250]
    """losses 的 shape 是 250"""
    for k in range(eval_iters):
      X, Y = get_batch(split)
      """X 與 Y 的 shape 都是 [4,8]"""
      logits, loss = model(X, Y)
      """logits 的 shape 是 [32,83], loss 的 shape 是 []"""
      """logits 在此似乎沒有功用"""
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [43]:
tab = nn.Embedding(vocab_size, vocab_size)

In [41]:
indx = torch.randint(1,(1,1), dtype=torch.long)
indx

tensor([[0]])

In [52]:
ls = torch.randn((5,3))
ls

tensor([[ 0.0376,  1.9811, -0.4050],
        [ 0.0323,  2.8938, -0.6420],
        [-0.7401,  0.4022,  1.3449],
        [ 0.5876, -0.0132, -0.5787],
        [ 0.5533, -1.6860,  1.0879]])

In [7]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) ## 輸入: vocab_size, 輸出: vocab_size

    def forward(self, index, targets=None):
        """index 的 shape 是 [B,T]"""
        logits = self.token_embedding_table(index)
        """logits 的 shape 是 [B,T,C]"""

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            """輸出logits 的 shape 是 [B*T, C]"""
            targets = targets.view(B*T)
            """targets 的 shape 是 [B*T]"""
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        """index 的 shape 是 [B,T]"""
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            """logists 的 shape 是 [B,T,C]"""
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            """logists 的 shape 是 [B,C]"""
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            """probs 的 shape 是 [B,C]"""
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            """index_next 的 shape 是 [B,1]"""
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
            """index 的 shape 是 [B,T+1] <= [B, T] + [B, 1] 在 dim=1"""
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)         ## tensor([[0]])
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
## m.generate(context, max_new_tokens=500) 的 shape 是 [1, 501]
print(generated_chars)



y?Y*?q8Qs]9!18WIl57vdBBV)NK"2b257W"RAOFN-rpwqp5tue!gYk?;Ct_Tm6j]j&E]P)u920mW7﻿JQxJ.fYSq﻿[jMB]XjyC[q7PS17lvj&﻿lH&OkKjm:q?GeI"2?DjE-f.wP Z:;vx)ialQ9kL.(K6/Xqhl_0Ars]7Q8YTjbYTEb#34 u(S85e f_n.kw!15
xz4WE:NDnpMDA/sut)﻿﻿?wHI)﻿r;v!IaTm'Qn521Vq7AM,T 9QN'W)﻿v"2Q
﻿?(PS:,7k*'v"f4﻿A/
,cWB7kQ92V]UGFn6PXw[U"Z9PUmkSypC﻿B;dfcPfD?eY*H':qw5T]PcGdO.jbolsi85M&V0A
CJytf;cSEyBm'ydL0U4WYeLpU]2﻿U*LAJ4)8QcOS&Iwb[LA*9Y m'g.0]f#cU"_tXAE6H*G.X;)
NLU?VU;],Ta"OkW;vVP[GS8rhBr&j-
;2SEt_pQ7
1O6J?GFS((﻿LU]4*:SEDcI,QPXkSEO" NK!E


In [8]:
losses = estimate_loss()
losses

{'train': tensor(5.0936), 'val': tensor(5.0711)}

In [22]:
for iter in range(max_iters): ## 0~999  ## 此 loop 可以執行多次,而optimizer設定可以只執行一次,也可以執行多次
    if iter % eval_iters == 0:
        losses = estimate_loss()
        """losses 是 dict 類型, 樣態是 {'train': 值, 'val': 值}"""
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')
    """xb 與 yb 的 shape 都是 [4,8]"""

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    """logits 的 shape 是 [32,83], loss 的 shape 是 []"""
    loss.backward()
    optimizer.step()
    optimizer.zero_grad(set_to_none=True)
print(loss.item())

step: 0, train loss: 3.144, val loss: 3.139
step: 250, train loss: 3.115, val loss: 3.127
step: 500, train loss: 3.091, val loss: 3.089
step: 750, train loss: 3.058, val loss: 3.086
2.975935697555542


In [17]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters): ## 0~999  ## 此 loop 可以執行多次,而optimizer設定可以只執行一次,也可以執行多次
    if iter % eval_iters == 0:
        losses = estimate_loss()
        """losses 是 dict 類型, 樣態是 {'train': 值, 'val': 值}"""
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')
    """xb 與 yb 的 shape 都是 [4,8]"""

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    """logits 的 shape 是 [32,83], loss 的 shape 是 []"""
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.097, val loss: 4.097
step: 250, train loss: 4.050, val loss: 4.053
step: 500, train loss: 3.994, val loss: 3.985
step: 750, train loss: 3.989, val loss: 3.956
3.987161159515381


* <font color="red">訓練的`for`大概需執行15次以上,才能讓loss降至3.0左右。</font>

***need to familiarize audience with optimizers (AdamW, Adam, SGD, MSE…) no need to jump into the formulas, just what the optimizer does for us and some of the differences/similarities between them***

1. **Mean Squared Error (MSE)**: MSE is a common loss function used in regression problems, where the goal is to predict a continuous output. It measures the average squared difference between the predicted and actual values, and is often used to train neural networks for regression tasks.
2. **Gradient Descent (GD):**  is an optimization algorithm used to minimize the loss function of a machine learning model. The loss function measures how well the model is able to predict the target variable based on the input features. The idea of GD is to iteratively adjust the model parameters in the direction of the steepest descent of the loss function
3. **Momentum**: Momentum is an extension of SGD that adds a "momentum" term to the parameter updates. This term helps smooth out the updates and allows the optimizer to continue moving in the right direction, even if the gradient changes direction or varies in magnitude. Momentum is particularly useful for training deep neural networks.
4. **RMSprop**: RMSprop is an optimization algorithm that uses a moving average of the squared gradient to adapt the learning rate of each parameter. This helps to avoid oscillations in the parameter updates and can improve convergence in some cases.
5. **Adam**: Adam is a popular optimization algorithm that combines the ideas of momentum and RMSprop. It uses a moving average of both the gradient and its squared value to adapt the learning rate of each parameter. Adam is often used as a default optimizer for deep learning models.
6. **AdamW**: AdamW is a modification of the Adam optimizer that adds weight decay to the parameter updates. This helps to regularize the model and can improve generalization performance. We will be using the AdamW optimizer as it best suits the properties of the model we will train in this video.

find more optimizers and details at torch.optim

In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


tesle?uKvmot spY
eoyErdB
xxfrem[o V8WLDl8Mkm'uz"Cdd
datys?1V!ourM5Cxz&THG8U2K.L_u.HLORuze ls lXtlel8M5J'B&RL)la.O16
Tb sL7&or
BW


 ithN:wk"th!OF :efaSB!!ardgn0kl pcly,[x0W5vSwooyw5.be:WP1k
,com﻿eard["f7&p9P0U0&Q6HWxR'[sizQ4C4,:"MK-R6YCvulexk0M5﻿6yEucoatallm.R"Sb"KKacop(6-1BS.L:GemsitQ4UL"magg.QHLG(0azar lPq'bzzgQBL::Bj7jneabM4VSQLar tud. PO5w]uldls8'pgv!BU3T7L*UHPokrriZ3-(. hY38G
ZdmpuVJns. dos"DomD.41,﻿HPPJJ)P24   gqplazsh.9c QyjM5O
d,:7Gx3gnaH,"w1﻿*"RLpPtWY_﻿,A
Srd a, ;diZKImThere?BSmJXwo Qnz
